# AP(P)3RO PROJECT (model de démo) #


#### M. Le Gras, A. Naullet, A. Calixte, P. Beaunieux ####

## Partie 1: Cartographie de la ville par le drone ##

On simule une ville avec un petit graphe

In [68]:
import numpy as np

In [69]:
#voici un graph simulant un morceau de ville
#(node1,node2, distance)                                                                                                                                 
G = [(1,2,29), (2,3,13), (3,4,21), (4,8,10), (8,2,2), (8,5,20), (8,6,12), (6,7,6), (6,1,22)]
print(G)

[(1, 2, 29), (2, 3, 13), (3, 4, 21), (4, 8, 10), (8, 2, 2), (8, 5, 20), (8, 6, 12), (6, 7, 6), (6, 1, 22)]


<img src="assets/img/graph_num.png" width="400">

### 1.2 Calcul du parcours le plus efficace pour le drone ###

L'idée c'est de trouver un chemin eulerien, cependant notre graphe n'est pas eulerien
Nous allons alors utilisé un algo qui transforme un graph non-eulerien en un graph eulerien (sans altéré ses propriétés). Enfin on pourra utiliser un algo qui trouve un chemin eulerien c-a-d un chemin qui passe par toutes les arretes du graphes (simulant les rues de la ville)

In [70]:
#ICI on fait l'algo du cycle_eulerien_avec_erreur

In [86]:
#fix le graph vers un graph eulerien
def odd_vertices(n, edges):
    deg = [0] * n
    for (a,b,c) in edges:
        deg[a] += 1
        deg[b] += 1
        
    return [a for a in range(n) if deg[a] % 2]

def is_edges(n, edges, node1, node2):
    for (a, b, c) in edges:
        if (a == node1 and b == node2) or (a == node2 and b == node1):
            return (a,b,c)
    return None

def list_edges(n, edges, l_vodd):
    res = []
    for a in l_vodd:
        for b in l_vodd:
            if (a == b):
                continue
            tmp = is_edges(n, edges, a, b)
            if (tmp and tmp not in res) :
                res.append(tmp)               
    return res

def shortest_edge_idx(l_edges):
    if (len(l_edges) == 0):
        return None
    shortest = l_edges[0][2]
    shortest_index = 0
    
    index = 1
    for (a,b,c) in l_edges[1:]:
        if (c < shortest):
            shortest = c
            shortest_index = index
        index+=1
    return l_edges[shortest_index]

def transform_to_eulerian(n, edges):
    list_vodd = odd_vertices(n, edges)
    l_edges = list_edges(n, edges, list_vodd)
    
    len_vodd = len(list_vodd)
    if (len_vodd == 2 or len_vodd == 0): #case is already eulerian
        return edges
    #print(list_vodd)
    #print(l_edges)
    while(len(list_vodd) != 2):
        shortest_edge = shortest_edge_idx(l_edges)
        #add edge between two
        (a,b,c) = shortest_edge
        new_edge = (b,a,c)
        edges.append(new_edge)
        
        #delete from vodd
        list_vodd.remove(a)
        list_vodd.remove(b)
        
        #delete from list_edges
        l_edges.remove(shortest_edge)
    return edges
#[(2, 1), (1, 6), (6, 7), (7, 6), (6, 8), (8, 4), (4, 3), (3, 2), (2, 8), (8, 5)]
#[(2, 1), (1, 6), (6, 7), (7, 6), (6, 8), (8, 4), (4, 3), (3, 2), (2, 8), (8, 5)]

In [72]:
# L'algo de transformation en graph eulerien doit donner le graph ci-dessous

<img src="assets/img/eulerian_graph.png" width="400">

In [113]:
G = [(1,2,29), (2,3,13), (3,4,21), (4,8,10), (8,2,2), (8,5,1), (8,6,12), (6,7,6), (6,1,22)]   
G = transform_to_eulerian(len(G), G)
G2 = []

### on enlève la ponderation
for (a,b,c) in G:
    G2.append((a,b))

In [114]:
from copy import copy

'''
    is_connected - Checks if a graph in the form of a dictionary is 
    connected or not, using Breadth-First Search Algorithm (BFS)
'''
def is_connected(G):
    start_node = list(G)[0]
    color = {v: 'white' for v in G}
    color[start_node] = 'gray'
    S = [start_node]
    while len(S) != 0:
        u = S.pop()
        for v in G[u]:
            if color[v] == 'white':
                color[v] = 'gray'
                S.append(v)
            color[u] = 'black'
    return list(color.values()).count('black') == len(G)

'''
    odd_degree_nodes - returns a list of all G odd degrees nodes
'''
def odd_degree_nodes(G):
    odd_degree_nodes = []
    for u in G:
        if len(G[u]) % 2 != 0:
            odd_degree_nodes.append(u)
    return odd_degree_nodes

'''
    from_dict - return a list of tuples links from a graph G in a 
    dictionary format
'''    
def from_dict(G):
    links = []
    for u in G:
        for v in G[u]:
            links.append((u,v))
    return links

'''
    fleury(G) - return eulerian trail from graph G or a 
    string 'Not Eulerian Graph' if it's not possible to trail a path
'''
def fleury(G):
    '''
        checks if G has eulerian cycle or trail
    '''
    odn = odd_degree_nodes(G)
    #print(odn)
    if len(odn) > 2 or len(odn) == 1:
        return 'Not Eulerian Graph'
    else:
        g = copy(G)
        trail = []
        if len(odn) == 2:
            u = odn[0]
        else:
            u = list(g)[0]
        while len(from_dict(g)) > 0:
            current_vertex = u
            for u in g[current_vertex]:
                g[current_vertex].remove(u)
                g[u].remove(current_vertex)
                bridge = not is_connected(g)
                if bridge:
                    g[current_vertex].append(u)
                    g[u].append(current_vertex)
                else:
                    break
            if bridge:
                g[current_vertex].remove(u)
                g[u].remove(current_vertex)
                g.pop(current_vertex)
            trail.append((current_vertex, u))
    return trail

def to_dict(G):
    dict_graph = {}
    for (a,b) in G:
        dict_graph[a] = []
        dict_graph[b] = []
    for (a,b) in G:
        dict_graph[a].append(b)
        dict_graph[b].append(a)
    return dict_graph

dict_G2 = to_dict(G2)
print(dict_G2)
print(fleury(dict_G2))

{1: [2, 7, 6, 6], 2: [1, 3, 8], 3: [2, 4], 4: [3, 8], 8: [4, 2, 5, 6], 5: [8], 6: [8, 7, 1, 1], 7: [6, 1]}
[(2, 1), (1, 7), (7, 6), (6, 1), (1, 6), (6, 8), (8, 4), (4, 3), (3, 2), (2, 8), (8, 5)]


In [115]:
# ICI ON AFFICHE le parcours

### 1.3 Recuperation des données du drones (simulation d'eneigement) ###

In [7]:
#ICI on simule l'eneigement (baill de plustard)